# Simulation Runner

Run example simulation in a loop 

In [1]:
import os
os.chdir('../')
import sys
import logging
import time
from datetime import date
import simpy

import pandas as pd

sys.path.insert(0, os.path.abspath('../../thesis_experiments'))
sys.path.insert(0, os.path.abspath('../../topsim_pipelines'))
sys.path.insert(0, os.path.abspath('../../shadow'))

# Framework defined models
from topsim.core.simulation import Simulation
from topsim.core.delay import DelayModel

# User defined models 
from user.telescope import Telescope # Instrument
from user.schedule.dynamic_plan import DynamicAlgorithmFromPlan # Scheduling
from user.schedule.greedy import GreedyAlgorithmFromPlan # Scheduling
from user.schedule.batch_allocation import BatchProcessing
from user.plan.batch_planning import BatchPlanning # Planning
from user.plan.static_planning import SHADOWPlanning 


logging.basicConfig(level="INFO")
logger = logging.getLogger()

STATIC_PLANNING_ALGORITHMS = ['heft', 'fcfs']

GLOBAL_SIM = pd.DataFrame()
GLOBAL_TASKS = pd.DataFrame()

DATE = str(date.today())

In [2]:
cfg = '2021_isc-hpc/config/single_size/40cluster/mos_sw10.json'
env = simpy.Environment()
timestamp = f'{time.time()}'.split('.')[0]
simulation = Simulation(
    env=env,
    config=cfg,
    instrument=Telescope,
    planning_algorithm='batch',
    planning_model=BatchPlanning('batch'),
    scheduling=BatchProcessing,
    delay=None,
    timestamp={timestamp}
)
sim, tasks = simulation.start()


INFO:user.telescope:telescope is now using 36 arrays
INFO:user.telescope:Telescope is now using 0 arrays
INFO:user.telescope:telescope is now using 18 arrays
INFO:user.telescope:Telescope is now using 0 arrays
INFO:user.telescope:telescope is now using 18 arrays
INFO:user.schedule.batch_allocation:Provisioning 17 resources to emu1
INFO:user.telescope:Telescope is now using 0 arrays
INFO:user.telescope:telescope is now using 36 arrays
INFO:user.schedule.batch_allocation:Provisioning 17 resources to emu2
INFO:user.schedule.batch_allocation:Provisioning 16 resources to dingo
INFO:user.telescope:Telescope is now using 0 arrays
INFO:user.schedule.batch_allocation:Provisioning 17 resources to wallaby
INFO:topsim.core.scheduler:emu1 Removed from Queue @2487
INFO:topsim.core.scheduler:emu2 Removed from Queue @2714
INFO:topsim.core.scheduler:dingo Removed from Queue @2770
INFO:topsim.core.scheduler:wallaby Removed from Queue @2813
INFO:topsim.core.simulation:Simulation Finished @ 2815.0


In [14]:
simulation.scheduler.algorithm._provision_resources

<bound method BatchProcessing._provision_resources of BatchProcessing>

In [ ]:
# RUNNING SIMULATION AND GENERATING DATA

for algorithm in algorithms:
    for config in sorted(os.listdir(
            '2021_isc-hpc/config/single_size/40cluster')):
        if '.json' in config:
            CONFIG = f'2021_isc-hpc/config/single_size/40cluster/{config}'
            env = simpy.Environment()
            instrument = Telescope
            timestamp = f'{time.time()}'.split('.')[0]
            simulation = Simulation(
                env=env,
                config=CONFIG,
                instrument=instrument,
                planning=algorithm,
                scheduling=GreedyAlgorithmFromPlan,
                delay=None,
                timestamp={timestamp}
            )
            sim, tasks = simulation.start()
            global_sim = global_sim.append(sim)
            global_tasks = global_tasks.append(tasks)
            print(algorithm, config, len(sim))
global_sim.to_pickle(f'{date}_global_sim_{}.pkl')
global_tasks.to_pickle(f'{date}_global_tasks_{}.pkl')